In [0]:
#from __future__ import print_function
import keras
from keras.datasets import fashion_mnist
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Model
from keras.models import Sequential

from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.datasets import cifar10
import numpy as np
import os, time

Using TensorFlow backend.


In [0]:
#Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 10
height= 28
width=28
data_augmentation = False
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

In [0]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
    
# #split into train and validation
# (x_train, x_val, y_train, y_val) = train_test_split(x_train, y_train, test_size = 0.15, random_state = 0)
# print("shapes after splitting ", x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)
    
#one hot encoding
y_train= to_categorical(y_train, num_classes) 
# y_val= to_categorical(y_val, num_classes) 
y_test= to_categorical(y_test, num_classes)

print("after hot encoding", y_train.shape, y_test.shape)
    
#reshape data
x_train = x_train.reshape(x_train.shape[0], height, width, 1)
x_test = x_test.reshape(x_test.shape[0], height, width, 1)
# x_val = x_val.reshape(x_val.shape[0], height, width, 1)

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

after hot encoding (60000, 10) (10000, 10)


In [0]:
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same', activation = 'relu', kernel_initializer='he_normal',  input_shape=(28,28,1) ) )
model.add(Dropout(0.15))
model.add(Conv2D(32, (3, 3), padding='same', activation = 'relu', kernel_initializer='he_normal' )) 
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu', kernel_initializer='he_normal' )) 
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Flatten())
    
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.10))

model.add(Dense(10, activation = 'softmax'))
   

W0704 07:04:59.900972 140192682633088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0704 07:04:59.918806 140192682633088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0704 07:04:59.922251 140192682633088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0704 07:04:59.943669 140192682633088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0704 07:04

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

W0704 07:06:02.773678 140192682633088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    start_time= time.time()
    history= model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              shuffle=True,
              )
    end_time= time.time()
    print("testing time", (end_time-start_time))
    
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    start_time= time.time()
    history=model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                
                        epochs=epochs, verbose=1, workers=4,
                        )
    end_time= time.time()
    print("testing time", (end_time-start_time))
   

W0704 07:06:06.657441 140192682633088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Not using data augmentation.
Epoch 1/10
60000/60000 [==============================] - 348s 6ms/step - loss: 0.5006 - acc: 0.8219
Epoch 2/10
60000/60000 [==============================] - 347s 6ms/step - loss: 0.3165 - acc: 0.8880
Epoch 3/10
60000/60000 [==============================] - 347s 6ms/step - loss: 0.2694 - acc: 0.9036
Epoch 4/10
60000/60000 [==============================] - 348s 6ms/step - loss: 0.2382 - acc: 0.9129
Epoch 5/10
60000/60000 [==============================] - 365s 6ms/step - loss: 0.2148 - acc: 0.9210
Epoch 6/10
60000/60000 [==============================] - 354s 6ms/step - loss: 0.2022 - acc: 0.9261
Epoch 7/10
60000/60000 [==============================] - 352s 6ms/step - loss: 0.1867 - acc: 0.9313
Epoch 8/10
60000/60000 [==============================] - 351s 6ms/step - loss: 0.1747 - acc: 0.9365
Epoch 9/10
60000/60000 [==============================] - 353s 6ms/step - loss: 0.1635 - acc: 0.9394
Epoch 10/10
60000/60000 [==============================] - 358

In [0]:
# Score trained model.
start_time= time.time()
scores = model.evaluate(x_test, y_test, verbose=1)
end_time= time.time()
print("testing time", (end_time-start_time))
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 10s 1ms/step
testing time 10.492141008377075
Test loss: 0.2255352610707283
Test accuracy: 0.9216


In [0]:
#Training history visualization

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: ignored